In [1]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import json
import warnings
warnings.filterwarnings('ignore')
#from label_decoders import *

In [2]:
config = json.load(open('settings.json'))
train = pd.read_csv(config['train'])
test = pd.read_csv(config['test'])


In [4]:
#all_data = train.append(test)
all_data=all_data.iloc[1:101]

In [6]:
all_data['Product_Info_2_char'] = all_data.Product_Info_2.str[0]
all_data['Product_Info_2_num'] = all_data.Product_Info_2.str[1]
all_data['Product_Info_2'] = pd.factorize(all_data['Product_Info_2'])[0]
all_data['Product_Info_2_char'] = pd.factorize(all_data['Product_Info_2_char'])[0]
all_data['Product_Info_2_num'] = pd.factorize(all_data['Product_Info_2_num'])[0]
all_data['BMI_Age'] = all_data['BMI'] * all_data['Ins_Age']
med_keyword_columns = all_data.columns[all_data.columns.str.startswith('Medical_Keyword_')]
all_data['Med_Keywords_Count'] = all_data[med_keyword_columns].sum(axis=1)
all_data.apply(lambda x: sum(x.isnull()),1)
all_data['countna'] = all_data.apply(lambda x: sum(x.isnull()),1)
all_data.fillna(-1, inplace=True)
all_data['Response'] = all_data['Response'].astype(int)

In [7]:
train_ohd = all_data[all_data['Response']>0].copy()
test_ohd = all_data[all_data['Response']<1].copy()

In [8]:
features=train_ohd.columns.tolist()
features = [x.replace('=','_') for x in features]
features = [x.replace('_','i') for x in features]
train_ohd.columns = features
features_t=test_ohd.columns.tolist()
features_t = [x.replace('=','i') for x in features_t]
features_t = [x.replace('_','i') for x in features_t]
test_ohd.columns = features_t

In [9]:
features.remove("Id")
features.remove("Response")

In [10]:
train_ohd['rf1'] = [0]*train_ohd.shape[0]
train_ohd['rf2'] = [0]*train_ohd.shape[0]
train_ohd['rf3'] = [0]*train_ohd.shape[0]
train_ohd['rf4'] = [0]*train_ohd.shape[0]
train_ohd['rf5'] = [0]*train_ohd.shape[0]
train_ohd['rf6'] = [0]*train_ohd.shape[0]
train_ohd['rf7'] = [0]*train_ohd.shape[0]
train_ohd['rf8'] = [0]*train_ohd.shape[0]
train_ohd['rf9'] = [0]*train_ohd.shape[0]
train_ohd['rf10'] = [0]*train_ohd.shape[0]
train_ohd['rf11'] = [0]*train_ohd.shape[0]
train_ohd['rf12'] = [0]*train_ohd.shape[0]
train_ohd['rf13'] = [0]*train_ohd.shape[0]

In [11]:
l = train_ohd.shape[0]
ind_list = [(range(0,l//10), list(filter(lambda x: x not in range(0,l//10), range(0,l)))), 
            (range(l//10,l//10*2), list(filter(lambda x: x not in range(l//10,l//10*2), range(0,l)))),
            (range(l//10*2,l//10*3), list(filter(lambda x: x not in range(l//10*2,l//10*3), range(0,l)))),
            (range(l//10*3,l//10*4), list(filter(lambda x: x not in range(l//10*3,l//10*4), range(0,l)))),
            (range(l//10*4,l//10*5), list(filter(lambda x: x not in range(l//10*4,l//10*5), range(0,l)))),
            (range(l//10*5,l//10*6), list(filter(lambda x: x not in range(l//10*5,l//10*6), range(0,l)))),
            (range(l//10*6,l//10*7), list(filter(lambda x: x not in range(l//10*6,l//10*7), range(0,l)))),
            (range(l//10*7,l//10*8), list(filter(lambda x: x not in range(l//10*7,l//10*8), range(0,l)))),
            (range(l//10*8,l//10*9), list(filter(lambda x: x not in range(l//10*8,l//10*9), range(0,l)))),
            (range(l//10*9,l), list(filter(lambda x: x not in range(l//10*9,l), range(0,l))))]

In [12]:
def labels_decoder1(lb):
    lb = [1 if x == 1 else 0 for x in lb]    
    return lb


def labels_decoder2(lb):
    lb = [1 if x == 2 else 0 for x in lb]    
    return lb


def labels_decoder3(lb):
    lb = [1 if x == 3 else 0 for x in lb]    
    return lb


def labels_decoder4(lb):
    lb = [1 if x == 4 else 0 for x in lb]    
    return lb


def labels_decoder5(lb):
    lb = [1 if x == 5 else 0 for x in lb]    
    return lb


def labels_decoder6(lb):
    lb = [1 if x == 6 else 0 for x in lb]    
    return lb


def labels_decoder7(lb):
    lb = [1 if x == 7 else 0 for x in lb]    
    return lb 


def labels_decoder8(lb):
    lb = [1 if x == 8 else 0 for x in lb]    
    return lb


def labels_decoder9(lb):
    lb = [1 if x < 3 else 0 for x in lb]    
    return lb


def labels_decoder10(lb):
    lb = [1 if x < 4 else 0 for x in lb]    
    return lb


def labels_decoder11(lb):
    lb = [1 if x < 5 else 0 for x in lb]    
    return lb


def labels_decoder12(lb):
    lb = [1 if x < 6 else 0 for x in lb]    
    return lb


def labels_decoder13(lb):
    lb = [1 if x < 7 else 0 for x in lb]    
    return lb

In [13]:
ld = [labels_decoder1,labels_decoder2,labels_decoder3,labels_decoder4,labels_decoder5,labels_decoder6,labels_decoder7,
      labels_decoder8,labels_decoder9,labels_decoder10,labels_decoder11,labels_decoder12,labels_decoder13]



In [14]:
i = 0
for l in ld:
    i = i + 1    
    
    for j in range(10):
        
        X_1, X_2 = ind_list[j][1], ind_list[j][0]
        y_1, y_2 = train_ohd.iloc[X_1]['Response'], train_ohd.iloc[X_2]['Response']
        
        rf = RandomForestClassifier(n_estimators=500, random_state=1)
        rf.fit(train_ohd[features].iloc[X_1],l(y_1))
        train_ohd['rf%s' % (i)].iloc[X_2] = rf.predict_proba(train_ohd[features].iloc[X_2]).T[1]

In [16]:
train_ohd.to_csv(config['train_rf'],index=0)


In [17]:
y = train_ohd['Response']

i = 0
for l in ld:
    i = i + 1    

###1
    rf = RandomForestClassifier(n_estimators=500, random_state=1)
    rf.fit(train_ohd[features],l(y))
    test_ohd['rf%s' % (i)] = rf.predict_proba(test_ohd[features]).T[1]

test_ohd.to_csv(config['test_rf'],index=0)

ValueError: Found array with 0 sample(s) (shape=(0, 131)) while a minimum of 1 is required.